In [1]:
import os
import pandas as pd
import numpy as np


In [2]:
# data loader
def load_data(folders):
    """ load csvs as flattened feature vectors with labels """
    data, labels = [], []

    for folder, _ in folders:
        if os.path.exists(folder):
            for file in os.listdir(folder):
                if file.endswith('.csv'):
                    file_path = os.path.join(folder, file)
                    df = pd.read_csv(file_path)
                    label = df.iloc[0, 0]  # extract label
                    features = df.iloc[:, 1:].values.flatten()  # flatten features
                    data.append(features)
                    labels.append(label)
        else:
            print(f"warning: folder {folder} not found.")

    return np.array(data), np.array(labels)

In [ ]:
# define training folders
train_folders = [
    ('ratdanceCSV/train', 1),  # label 1 for ratdance
    ('negative_controlCSV/train', 0)  # label 0 for negative control
]

# load dataset
X, y = load_data(train_folders)

# normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)
